## Markov chains

We'll cover the following topics in this notebook:
- Random number generation
- Model of finite state machines
- Model of Markov chains


### Random number generation

In [ ]:
let seed = 0
let () = Random.init seed

let _ = Random.int 10
let ls = List.init 10 (fun _ -> Random.int 10)

(* Simple simulation *)
let compute_prob total =
	let counts = Array.make 10 0. in
	let samples = List.init total (fun _ -> Random.int 10) in
	let rec aux = function
		| [] -> ()
		| x :: xs -> 
			counts.(x) <- counts.(x) +. 1.;
			aux xs
	in 
		aux samples;
		Array.map (fun x -> x /. (float_of_int total)) counts

### Finite state machines

In [ ]:
type states = A | B

let start = B

let accept = function
	| A -> false
	| B -> true

let transit s c = match (s, c) with
	| A, 'a' -> A
	| A, 'b' -> B
	| B, 'a' -> A
	| B, 'b' -> B
	| _ -> raise (Invalid_argument "Unexpected character!")

let match_str str = 
	let len = String.length str in
	let ls = List.init len (fun i -> String.get str i) in
	let rec matching state = function
		| [] -> accept state
		| c :: cs -> 
			let state' = transit state c in
			matching state' cs
	in

		matching start ls

let _ = match_str "abababba"

In [ ]:
type states = A0 | A1 | A2 | Acc

let start = A0

let accept = function
	| Acc -> true
	| _ -> false

let transit s c = match (s, c) with
	| A0, 'a' -> A1
	| A1, 'a' -> A2
	| A2, 'a' -> Acc
	| Acc, 'a' -> Acc
	| state, 'b' -> state
	| _ -> raise (Invalid_argument "Unexpected character!")

let match_str str = 
	let len = String.length str in
	let ls = List.init len (fun i -> String.get str i) in
	let rec matching state = function
		| [] -> accept state
		| c :: cs -> 
			let state' = transit state c in
			matching state' cs
	in

		matching start ls

let _ = match_str "abababba"

### Markov chains

In [ ]:
type states = A | B

let next_state current = 
	let p = Random.float 1.0 in
	match current with
		| A -> if p <= 0.5 then A else B
		| B -> if p <= 0.5 then A else B

let trace init steps = 
		let current = ref init in
		let walk () = 
			current := next_state (!current);
			!current
		in
			List.init steps (fun _ -> walk())

let dist init steps = 
	let counts = [|0.; 0.|] in
	let tr = trace init steps in
	let rec aux = function
		| [] -> ()
		| A :: xs -> counts.(0) <- counts.(0) +. 1.; aux xs
		| B :: xs -> counts.(1) <- counts.(1) +. 1.; aux xs
	in
		aux tr;
		Array.map (fun x -> x /. (float_of_int steps)) counts

In [ ]:
type states = S0 | S1 | S2 | Crit

let next_state current = 
	let p = Random.int 4 in
	match current with
		| S0 -> if p = 3 then S0 else S1 
		| S1 -> if p = 3 then S0 else S2
		| S2 -> if p = 3 then S0 else Crit
		| Crit -> S0

let trace init steps = 
		let current = ref init in
		let walk () = 
			current := next_state (!current);
			!current
		in
			List.init steps (fun _ -> walk())
	
let dist init steps = 
	let counts = [|0.; 0.; 0.; 0.|] in
	let tr = trace init steps in
	let rec aux = function
		| [] -> ()
		| S0 :: xs -> counts.(0) <- counts.(0) +. 1.; aux xs
		| S1 :: xs -> counts.(1) <- counts.(1) +. 1.; aux xs
		| S2 :: xs -> counts.(2) <- counts.(2) +. 1.; aux xs
		| Crit :: xs -> counts.(3) <- counts.(3) +. 1.; aux xs
	in
		aux tr;
		Array.map (fun x -> x /. (float_of_int steps)) counts 

let prob steps = 
	let stable_dist = dist S0 steps in
	(stable_dist.(0) /. 3.) +.
	(stable_dist.(1) /. 3.) +.
	(stable_dist.(2) /. 3.) +.
	stable_dist.(3)

let actual = Array.map (fun x -> x /. 175.) [|64.; 48.; 36.; 27. |]